# Coreção dos Dados

In [ ]:
import pandas as pd
from datetime import datetime


In [ ]:
#Importando e corrigindo o nome das colunas
fazendeiros = pd.read_excel("planilhas/dadosFazendeiros.xlsx", engine = "openpyxl")
transbordo = pd.read_csv("planilhas/dadosTransbordo.csv", sep= ";")
fazendeiros.rename(columns = {'DAT.ENT':'data', 'HOR.DIG':'hora', 'NOME UNID.':'unidade', 'PESO2':'peso'}, inplace = True)
transbordo.rename(columns = {'DT ENT':'data', 'HORA':'hora', 'NOME DESTINO':'unidade', ' TAR DES ':'peso'}, inplace = True)


In [ ]:
# Selecionado apenas os dados importantes

florestaFazendeiros = fazendeiros[fazendeiros.unidade == "FLORESTA                 "]
florestaTransbordo = transbordo[transbordo.unidade == "FLORESTA"]
maringaFazendeiros = fazendeiros[fazendeiros.unidade == "FABRICA DE OLEO          "]
maringaTransbordo = transbordo[transbordo.unidade == "FABRICA DE OLEO"]

# Juntando os dados
print("Tamanho das tabelas:")
print("")
print("Floresta:")
print(f"Fazedeiro = {len(florestaFazendeiros)}")
print(f"Transbordo = {len(florestaTransbordo)}")
print(f"Total = {len(florestaTransbordo) + len(florestaFazendeiros)}")
print("")
print("Maringa")
print(f"Fazendeiro = {len(maringaFazendeiros)}")
print(f"Transbordo = {len(maringaTransbordo)}")
print(f"Total = {len(maringaTransbordo) + len(maringaFazendeiros)}")



### Correção dos dados de **data** e **hora** 

#### Dados do Transbordo

In [ ]:
#Função para correção da data
def to_date_costum(dt):
    data = ""
    try:
        stg = f"{dt[0]}{dt[1]}-{dt[3]}{dt[4]}-{dt[6]}{dt[7]}{dt[8]}{dt[9]}"
        data = datetime.strptime(stg, '%d-%m-%Y').date()
        return data
    except:
        data = "ERRO"
        return data

#Função para correção da hora

def to_hour_costum(dt):
    hora = ""
    try:
        hora = datetime.strptime(dt, '%H:%M').time()
        return hora
    except:
        hora = "ERRO"
        return hora

# Correção da data
maringaTransbordo['data'] = maringaTransbordo['data'].apply(lambda x: to_date_costum(x))
florestaTransbordo['data'] = florestaTransbordo['data'].apply(lambda x: to_date_costum(x))

# Correção da hora
maringaTransbordo['hora'] = maringaTransbordo['hora'].apply(lambda x: to_hour_costum(x))
florestaTransbordo['hora'] = florestaTransbordo['hora'].apply(lambda x: to_hour_costum(x))


In [ ]:
# Analise das linhas com erro na data
print(f"Erro na data em Maringa = {len(maringaTransbordo[maringaTransbordo.data == 'ERRO'])}")
print(f"Erro na data na Floresta = {len(florestaTransbordo[florestaTransbordo.data == 'ERRO'])}")
print(f"Erro na hora em Maringa = {len(maringaTransbordo[maringaTransbordo.hora == 'ERRO'])}")
print(f"Erro na hora na Floresta = {len(florestaTransbordo[florestaTransbordo.hora == 'ERRO'])}")

In [ ]:
#Remocao das linhas com erro
antesM = len(maringaTransbordo)
antesF = len(florestaTransbordo)
maringaTransbordo = maringaTransbordo[maringaTransbordo.data != 'ERRO']
florestaTransbordo = florestaTransbordo[florestaTransbordo.data != 'ERRO']
maringaTransbordo = maringaTransbordo[maringaTransbordo.hora != 'ERRO']
florestaTransbordo = florestaTransbordo[florestaTransbordo.hora != 'ERRO']

print(f"Linhas removidas de Maringa = {antesM - len(maringaTransbordo)}")
print(f"Linhas removidas da Floresta = {antesF - len(florestaTransbordo)}")



#### Dados dos Fazendeiros

In [ ]:
# Correção da data
maringaFazendeiros['data'] = maringaFazendeiros['data'].apply(lambda x: x.date())
florestaFazendeiros['data'] = florestaFazendeiros['data'].apply(lambda x: x.date())


In [ ]:
# Análise Maringá
print("<------------------- TRANSBORDO ------------------->")
print(type(maringaTransbordo.data.to_list()[0]))
print(type(maringaTransbordo.hora.to_list()[0]))
print(maringaTransbordo.head())
print("")
print("")

print("<------------------- FAZENDEIRO ------------------->")
print(type(maringaFazendeiros.data.to_list()[0]))
print(type(maringaFazendeiros.hora.to_list()[0]))
print(maringaFazendeiros.head())

In [ ]:
# Análise Floresta
print("<------------------- TRANSBORDO ------------------->")
print(type(florestaTransbordo.data.to_list()[0]))
print(type(florestaTransbordo.hora.to_list()[0]))
print(florestaTransbordo.head())
print("")
print("")

print("<------------------- FAZENDEIRO ------------------->")
print(type(florestaFazendeiros.data.to_list()[0]))
print(type(florestaFazendeiros.hora.to_list()[0]))
print(florestaFazendeiros.head())

#### Corrigindo os Pesos

In [ ]:
def corrigePeso(x):
    try:
        return float(x.replace(".", ""))
    except:
        return "ERRO"

maringaTransbordo['peso'] = maringaTransbordo['peso'].apply(lambda x: corrigePeso(x))
florestaTransbordo['peso'] = florestaTransbordo['peso'].apply(lambda x: corrigePeso(x))

#Remocao das linhas com erro
antesM = len(maringaTransbordo)
antesF = len(florestaTransbordo)
maringaTransbordo = maringaTransbordo[maringaTransbordo.peso != 'ERRO']
florestaTransbordo = florestaTransbordo[florestaTransbordo.peso != 'ERRO']

print(f"Linhas removidas de Maringa = {antesM - len(maringaTransbordo)}")
print(f"Linhas removidas da Floresta = {antesF - len(florestaTransbordo)}")

### Juntando as duas bases de dados

In [ ]:
# combina todos os dados de data e hora
datetimeFlorestaFazendeiros = []
for i in range(0,len(florestaFazendeiros)):
    temp1 = florestaFazendeiros.data.to_list()[i]
    temp2 = florestaFazendeiros.hora.to_list()[i]
    datetimeFlorestaFazendeiros.append(datetime.combine(temp1, temp2))

datetimeFlorestaTransbordo = []
for i in range(0,len(florestaTransbordo)):
    temp1 = florestaTransbordo.data.to_list()[i]
    temp2 = florestaTransbordo.hora.to_list()[i]
    datetimeFlorestaTransbordo.append(datetime.combine(temp1, temp2))

datetimeMaringaFazendeiros = []
for i in range(0,len(maringaFazendeiros)):
    temp1 = maringaFazendeiros.data.to_list()[i]
    temp2 = maringaFazendeiros.hora.to_list()[i]
    datetimeMaringaFazendeiros.append(datetime.combine(temp1, temp2))

datetimeMaringaTransbordo = []
for i in range(0,len(maringaTransbordo)):
    temp1 = maringaTransbordo.data.to_list()[i]
    temp2 = maringaTransbordo.hora.to_list()[i]
    datetimeMaringaTransbordo.append(datetime.combine(temp1, temp2))


In [ ]:
# Cria a coluna com data e hora nas databases
florestaFazendeiros.insert(1, "dataCerta", datetimeFlorestaFazendeiros)
florestaTransbordo.insert(1, "dataCerta", datetimeFlorestaTransbordo)
maringaFazendeiros.insert(1, "dataCerta", datetimeMaringaFazendeiros)
maringaTransbordo.insert(1, "dataCerta", datetimeMaringaTransbordo)

In [ ]:
#Junta as bases de Fazendeiros e de Transbordo
floresta = pd.concat([florestaFazendeiros, florestaTransbordo])
maringa = pd.concat([maringaFazendeiros, maringaTransbordo])

# Remove colunas desnecessárias
floresta = floresta[["dataCerta", "peso", "unidade"]]
maringa = maringa[["dataCerta", "peso", "unidade"]]
floresta.sort_values("dataCerta")
maringa.sort_values("dataCerta")



In [ ]:
# Exporta as tabelas finais
floresta.to_excel("planilhas/floresta.xlsx", )
maringa.to_excel("planilhas/maringa.xlsx")